# Tool Calling(Function calling)
`Tool Calling`を単独で実行する例。  
LLMに使用可能なツール情報を渡して実行すべきツールを決定させる。(LLM自体はツールの実行は行わない点に注意)

## パッケージインストール

In [2]:
%pip install -qU openai

Note: you may need to restart the kernel to use updated packages.


## 実装
`tools`というリクエストパラメータで使用可能なツール(今回は関数)一覧を渡すことにより、レスポンスとして質問に応じたツールを返す。  
使用すべき関数の情報はレスポンスの`tool_calls`に含まれる。  
`tool_choice`というパラメータにより、レスポンスとして呼び出す関数のみ(`tool_calls`)を返すか通常のテキストを返すかを制御できる。  
`tools`を指定すると`tool_choice`は`auto`になるため今回のような結果が得られる。 

- Function calling(https://platform.openai.com/docs/guides/function-calling/example-use-cases?api-mode=responses)

In [1]:

from openai import OpenAI

# toolsというリストにLLMが使用可能な関数を追加する(ここでは上で定義したget_weather()を登録)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "天候に関する情報を取得するツール",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "天候を調べる都市名",
                    },
                },
                "required": ["city"],
            },
        },
    }
]


client = OpenAI()

messages = [
    {"role": "user", "content": "東京の天気はどうですか？"} 
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,   # 使用可能な関数一覧
)

print(response.to_json(indent=2))  # LLMによりtoolsの中から使用すべき関数(get_current_weather)が選択される


{
  "id": "chatcmpl-BZAE7FTzbn6onFlD6o65PdnaQtAS5",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "tool_calls": [
          {
            "id": "call_kQAnd8WUvybyjhH8uxw55nXj",
            "function": {
              "arguments": "{\"city\":\"東京\"}",
              "name": "get_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1747720491,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_54eb4bd693",
  "usage": {
    "completion_tokens": 14,
    "prompt_tokens": 68,
    "total_tokens": 82,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_

## curlで実行する例
`curl`でOpenAIのAPIを直接実行してみる。  
`function_call`として選択したツール情報が返される。

事前に`OPENAI_API_KEY`を環境変数に設定しておく

```bash
curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-4o-mini",
    "messages": [
      {"role": "user", "content": "東京の天気はどうですか？"}
    ],
    "functions": [
      {
        "name": "get_weather",
        "description": "天候に関する情報を取得するツール",
        "parameters": {
          "type": "object",
          "properties": {
            "city": {
              "type": "string",
              "description": "天候を調べる都市名"
            }
          },
          "required": ["city"]
        }
      }
    ]
  }'
```

実行結果
```bash
{
  "id": "chatcmpl-BZDLeMH5Zw8d5bOwRH949oYEIQpFa",
  "object": "chat.completion",
  "created": 1747732490,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_weather",
          "arguments": "{\"city\":\"東京\"}"
        },
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 68,
    "completion_tokens": 14,
    "total_tokens": 82,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": "fp_54eb4bd693"
}
```